# Домашнее задание №1: линейная регрессия (максимум 10 баллов)

Некоторые задания будут по вариантам (всего 4 варианта). Чтобы выяснить свой вариант, посчитайте количество букв в своей фамилии, возьмете остаток от деления на 4 и прибавьте 1. (1 вариант у меня)


In [44]:
import numpy as np

## Многомерная линейная регрессия из sklearn

Создадим набор данных для многомерной регрессии

In [45]:
from sklearn.datasets import make_regression

X, y = make_regression(n_samples = 10000)
print(X.shape, y.shape)

(10000, 100) (10000,)


У нас 10000 объектов и 100 признаков. Для начала решим задачу аналитически "из коробки".

In [46]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

reg = LinearRegression().fit(X, y)
print(mean_squared_error(y, reg.predict(X)))

2.637260890980729e-25


Теперь попробуем обучить линейную регрессию методом градиентного спуска "из коробки"

In [47]:
from sklearn.linear_model import SGDRegressor
reg = SGDRegressor(alpha=0.00000001).fit(X, y)
print(mean_squared_error(y, reg.predict(X)))
reg.coef_

2.4376973793436887e-12


array([-7.40117592e-09,  7.14768035e+01,  1.62511378e-08, -5.62342799e-08,
       -2.29130650e-09,  3.39077435e-08,  4.18334580e+01,  1.79319250e-08,
        5.14434348e-09, -7.08661483e-09, -1.90969405e-08, -1.26304870e-08,
       -1.00619438e-09, -4.28319020e-08, -1.97705430e-08, -3.02065913e-08,
        2.70624736e+01,  4.72317206e-08, -4.05058294e-08,  1.11983847e+01,
       -4.14530581e-10, -1.94581872e-08, -1.51324123e-08,  7.91050637e-09,
       -7.10788408e-09, -1.26843500e-08, -1.73395388e-08, -1.26960054e-08,
       -4.00751865e-08,  1.63275494e+01, -5.73345490e-10,  2.72234409e-08,
       -3.98139723e-08,  1.77472378e-08,  6.41833147e-09, -6.28273121e-08,
        1.29095871e-08,  1.20595804e-08, -3.26734523e-09, -1.01178498e-08,
        6.11944223e+01, -1.43091637e-09, -2.61520464e-09,  2.27417925e-08,
        4.31882641e-08, -6.80324841e-08, -2.01062448e-08, -1.68857064e-08,
       -3.61060467e-08, -2.04470822e-08,  2.58472351e-08,  1.14209338e-08,
       -7.50637981e-09, -

***Задание 1 (1 балл).*** Объясните, чем вызвана разница в значениях двух полученных значений метрики?

***Задание 2 (1 балл).*** Подберите гиперпараметры в методе градиентного спуска так, чтобы значение MSE было близко к значению MSE, полученному при обучении LinearRegression.

(Задание 1) Разница в значениях MSE между LinearRegression и SGDRegressor вызвана тем, что:

LinearRegression даёт точное аналитическое решение, минимизирующее MSE на данных без шума.
SGDRegressor же из-за своей итеративной природы, возможной маленькой сходимости, наличия регуляризации и настройки параметров, не достигает такого же уровня минимизации MSE

Задание 2:

In [48]:
class LinearRegressionGD:
    def __init__(self, alpha=0.01, max_iter=10000, tol=1e-6):
        self.alpha = alpha
        self.max_iter = max_iter
        self.tol = tol

    def fit(self, X, y):
        self.theta = np.zeros(X.shape[1])
        self.mse_history = []
        for i in range(self.max_iter):
            y_pred = X.dot(self.theta)
            error = y_pred - y
            mse = np.mean(error ** 2)
            self.mse_history.append(mse)
            gradient = X.T.dot(error) / len(y)
            self.theta -= self.alpha * gradient
            if i > 0 and abs(self.mse_history[i] - self.mse_history[i-1]) < self.tol:
                break

    def predict(self, X):
        return X.dot(self.theta)

reg_gd = LinearRegressionGD(alpha=0.001, max_iter=10000, tol=1e-6)
reg_gd.fit(X, y)
print(np.mean((y - reg_gd.predict(X)) ** 2))


0.0005715668182927271


## Ваша многомерная линейная регрессия

***Задание 3 (5 баллов)***. Напишите собственную многомерную линейную регрессию, оптимизирующую MSE методом *градиентного спуска*. Для этого используйте шаблонный класс.

Критерий останова: либо норма разности весов на текущей и предыдущей итерациях меньше определенного значения (первый и третий варианты), либо модуль разности функционалов качества (MSE) на текущей и предыдущей итерациях меньше определенного значения (второй и четвертый варианты). Также предлагается завершать обучение в любом случае, если было произведено слишком много итераций.

***Задание 4 (2 балла)***. Добавьте l1 (первый и четвертый варианты) или l2 (второй и третий варианты) регуляризацию.

In [51]:
class LinearRegression(object):
    def __init__(self, alpha=0.01, l_ratio=0.0, tol=1e-6, max_iter=10000):
        '''
        Инициализация параметров модели:
        alpha - скорость обучения (learning rate)
        l_ratio - коэффициент L1-регуляризации
        tol - порог для критерия остановки по норме разности весов
        max_iter - максимальное количество итераций обучения
        '''
        self.alpha = alpha                  
        self.l_ratio = l_ratio            
        self.tol = tol           
        self.max_iter = max_iter           
        self.weights = None             

    def fit(self, X, y):
        '''
        Метод для обучения линейной регрессии:
        X - матрица признаков размерности (n_samples, n_features)
        y - вектор целевых значений размерности (n_samples,)
        '''
        n_samples, n_features = X.shape

        X = np.hstack([np.ones((n_samples, 1)), X])

        self.weights = np.zeros(n_features + 1)          
        weights_prev = np.full(n_features + 1, np.inf)   

        for iteration in range(self.max_iter):
            y_pred = np.dot(X, self.weights)

            error = y_pred - y
            gradient = (1 / n_samples) * np.dot(X.T, error) + self.l_ratio * np.sign(self.weights)

            self.weights -= self.alpha * gradient

            norm_diff = np.linalg.norm(self.weights - weights_prev)
            if norm_diff < self.tol:
                print(f'Сошлось за {iteration+1} итераций.')
                break

            weights_prev = self.weights.copy()
        else:
            print(f'Достигнуто максимальное количество итераций: {self.max_iter}')

    def predict(self, X):
        '''
        Метод для предсказания значений на новых данных:
        X - матрица признаков размерности (n_samples, n_features)
        '''
        n_samples = X.shape[0]
        X = np.hstack([np.ones((n_samples, 1)), X])
        return np.dot(X, self.weights)

In [52]:
my_reg = LinearRegression()
my_reg.fit(X, y)
assert mean_squared_error(y, my_reg.predict(X)) < 1e-3
print('You are amazing! Great work!')

Сошлось за 1576 итераций.
You are amazing! Great work!


***Задание 5 (1 балл)***. Обучите линейную регрессию из коробки с l1-регуляризацией (from sklearn.linear_model import Lasso, первый и четвертый варианты) или с l2-регуляризацией (from sklearn.linear_model import Ridge, второй и третий варианты) с значением параметра регуляризации 0.1. Обучите вашу линейную регрессию с тем же значением параметра регуляризации и сравните результаты. Сделайте выводы.

In [38]:
from sklearn.linear_model import Lasso

np.random.seed(0)
n_samples = 100
n_features = 5

X = np.random.randn(n_samples, n_features)
true_weights = np.array([1.5, -2.0, 0.0, 3.0, 0.0])
y = X @ true_weights + 4 + np.random.randn(n_samples) * 0.5 

my_reg = LinearRegression(alpha=0.01, l_ratio=0.1, tol=1e-6, max_iter=10000)
my_reg.fit(X, y)

lasso_reg = Lasso(alpha=0.1, max_iter=10000, tol=1e-6)
lasso_reg.fit(X, y)

my_preds = my_reg.predict(X)
lasso_preds = lasso_reg.predict(X)
my_mse = mean_squared_error(y, my_preds)
lasso_mse = mean_squared_error(y, lasso_preds)

print("Веса собственной линейной регрессии:")
print(my_reg.weights)

print("\nВеса линейной регрессии из sklearn (Lasso):")
print(np.hstack([lasso_reg.intercept_, lasso_reg.coef_]))

print(f"\nMSE собственной линейной регрессии: {my_mse}")
print(f"MSE линейной регрессии из sklearn (Lasso): {lasso_mse}")

Достигнуто максимальное количество итераций: 10000
Веса собственной линейной регрессии:
[ 3.78086714e+00  1.40987530e+00 -1.87224272e+00  4.69158115e-04
  2.88740319e+00  1.21507247e-05]

Веса линейной регрессии из sklearn (Lasso):
[ 3.88303711  1.42018829 -1.86972276  0.00659282  2.89537302 -0.        ]

MSE собственной линейной регрессии: 0.3005145337930805
MSE линейной регрессии из sklearn (Lasso): 0.2859302429787127
